# Unsupervised Clustering Model
A notebook that predicts loan grade for Problem 2 using unsupervised learning and K-means square

In [1]:
#Imports

#Data
import pandas as pd

#ML

#Other
import pickle

#General
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#knn
from sklearn.cluster import KMeans

In [3]:
#Getting the csv file's name with the wrangled data
data_file='wrang_xyz_data.csv'

#Splitting the data into different categories that make sense
loan_data=['purpose','initial_list_status','term','loan_amnt']
emp_data=['emp_length','collections_12_mths_ex_med','acc_now_delinq','home_ownership','annual_inc','verification_status','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','total_acc','earliest_cr_line','dti','tot_cur_bal','tot_coll_amt'] #address

#The following inputs are left out as they are only useful for problem 1.
#out=['last_pymnt_d','last_credit_pull_d','recoveries','collection_recovery_fee','last_pymnt_amnt','total_pymnt','total_rec_int','int_rate','out_prncp',''total_rec_late_fee','default_ind']

#Getting the features that will be included in the model
features=loan_data+emp_data

In [5]:
#Clustering unsupervised model predicting loan grade for problem 2
#The model's inputs are the data_file which should be set equal to the wrangled data file and the value to be predicted 
#(pred_value) which in this case is the 'grade'. It could also be the 'sub_grade'

def get_model_unsup(data_file='wrang_xyz_data.csv',pred_value=['grade']):
    #Imporitng the wrangled csv file and including the useful columns for it
  
    df = pd.read_csv('data/'+data_file,usecols=features+pred_value) #int_rate

    X=df.drop('grade',axis=1)

    X_scale=X.drop(['purpose','verification_status','home_ownership','initial_list_status','term'],axis=1)
    X_non_scale=X[['purpose','verification_status','home_ownership','initial_list_status','term']]

    #Scaling the data
    scaler = StandardScaler()
    scaler.fit(X_scale)
    #pickle.dump(scaler,open('scaler_unsup','wb'))
    X_scale = pd.DataFrame(scaler.transform(X_scale),index=X.index,columns=X_scale.columns)
    X=X_scale.join(X_non_scale)

    #One-hot Encoding
    ohe_cols=['purpose','verification_status','home_ownership','initial_list_status','term'] #address
    ohe = OneHotEncoder(handle_unknown='ignore')
    ohe.fit(df[ohe_cols])
    #pickle.dump(ohe,open('ohe_unsup','wb'))
    X_enc = pd.DataFrame(ohe.transform(X[ohe_cols]).toarray(),index=X.index)
    X=X.join(X_enc).drop(ohe_cols,axis=1)
    X.columns = X.columns.map(str)

    #Using the K-Means-Square algorith for clustering
    model_unsup = KMeans(n_clusters=7)
    model_unsup.fit(X)

    #Creating a function assigning a grade to a label
    def converter(cluster):
        clust={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}
        return clust[cluster]
        
    df['Cluster'] = df['grade'].apply(converter)

    #Printing useful metrics
    print(confusion_matrix(df['Cluster'],model_unsup.labels_))
    print(classification_report(df['Cluster'],model_unsup.labels_))

    return model_unsup
#Running the function 
model_unsup=get_model_unsup(pred_value=['grade'])
#pickle.dump(model_unsup,open('model_unsup','wb'))

[[43740 28657 42155 26226   136   999  3747]
 [79033 35583 75088 42785   924  3141 11442]
 [71429 34486 70497 43602  1324  3443 12069]
 [38041 19158 38623 27080   855  1998  7042]
 [17207 12056 17285 14990   477  1009  3422]
 [ 5241  4163  5402  4926   160   298  1138]
 [ 1082  1002  1185  1184    52    74   294]]
              precision    recall  f1-score   support

           0       0.17      0.30      0.22    145660
           1       0.26      0.14      0.19    247996
           2       0.28      0.30      0.29    236850
           3       0.17      0.20      0.18    132797
           4       0.12      0.01      0.01     66446
           5       0.03      0.01      0.02     21328
           6       0.01      0.06      0.01      4873

    accuracy                           0.21    855950
   macro avg       0.15      0.15      0.13    855950
weighted avg       0.22      0.21      0.20    855950

